<a href="https://colab.research.google.com/github/Atulchandil/atul/blob/main/capex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
import glob
import numpy as np
import pandas as pd
os.environ['OPENAI_API_KEY'] = '37a9009fee4f473fbc8d499ace9581fa'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://new-capex.openai.azure.com/'
import json
import os



In [51]:
!pip install langchain langchain-community openai chromadb tiktoken pypdf sentence_transformers AzureOpenAI langchain_openai PyPDF2 azure-core azure-search-documents langchain_core faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 39.3 MB/s eta 0:00:00


In [5]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.question_answering.chain import load_qa_chain
from langchain import LLMChain, PromptTemplate
from langchain.schema import HumanMessage
from sentence_transformers import SentenceTransformer
from langchain_community.vectorstores.chroma import Chroma
#model_name = "yiyanghkust/finbert-tone"
#embeddings = SentenceTransformerEmbeddings(model_name=model_name)
embeddings=SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L6-v2")





/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<ipython-input-5-1410fb11cd65>:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings=SentenceTransformerEmbeddings(model_name="paraphrase-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://hugg

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# persist_directory='/content/sample_data/chroma'
# model_name = "yiyanghkust/finbert-tone"
# embedding_model = SentenceTransformer(model_name)
# embeddings = SentenceTransformerEmbeddings(model_name=model_name)

# # Initialize Chroma without specifying embedding_dim
# vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

<ipython-input-4-cc2e24d20e05>:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)


In [5]:
# folder_path = '/content/sample_data/'
# files = glob.glob(os.path.join(folder_path, '**/*.pdf'), recursive=True)
# for fname in files:
#     document_loader = PyPDFLoader(fname)
#     documents = document_loader.load()
#     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
#     text = text_splitter.split_documents(documents)

#     for doc in text:
#         doc.metadata = {"file_name": os.path.basename(fname)}

#     vectordb.add_documents(documents=text)

# vectordb.persist()

<ipython-input-5-2d268684237e>:14: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [6]:
folder_path='/content/sample_data/'
persist_directory='/content/sample_data/embeddings'
files = glob.glob(os.path.join(folder_path, '**/*.pdf'), recursive=True)
for fname in files:
  document_loader=PyPDFLoader(fname)
  documents=document_loader.load()
  text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
  text=text_splitter.split_documents(documents)
  vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)
  for doc in text:
    doc.metadata = {"file_name": os.path.basename(fname)}
  vectordb.add_documents(documents=text)
vectordb.persist()

<ipython-input-6-f71727d686ad>:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)
<ipython-input-6-f71727d686ad>:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [7]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="capex-gpt4",
    model_name="gpt-4"
)

<ipython-input-7-3e117aba659c>:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  llm = AzureChatOpenAI(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:168: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://new-capex.openai.azure.com/ to https://new-capex.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/chat_models/azure_openai.py:175: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should

In [ ]:

# from langchain.llms import AzureOpenAI

# llm = AzureOpenAI(
#     deployment_name="capex-gpt35-1",
#     model_name="gpt-35-turbo-instruct"
# )

/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:883: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://capex-hackaton.openai.azure.com/ to https://capex-hackaton.openai.azure.com/openai.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:890: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_deployment`) is specified then `openai_api_base` (or alias `base_url`) should not be. Instead use `deployment_name` (or alias `azure_deployment`) and `azure_endpoint`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_community/llms/openai.py:898: UserWarning: As of openai>=1.0.0, if `openai_api_base` (or alias `base_url`) is specified it is expected to be of the form https://example-resource.azure.openai.com/openai/deployments/example-deployment. Updating htt

In [21]:
full_prompt='''
You are an AI assistant tasked with extracting capital expenditure (Capex) data from earnings call transcripts. CapEx refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, industrial buildings, or equipment. It is often found in the financial statements or discussed during earnings calls.

The name of the file has details like company name, year, and quarter. For example:
If the file name is AMZN-Q1-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q1.
If the file name is AMZN-Q2-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q2.
If the file name is CENCORA-Q2-2022-Earnings-Release, then the comany name is CENCORA, the year is 2022 and the quarter is Q2.


Always use the file name to extract company name, quarter, and year information.
Company name, quarter, and year information must be always be derived from file name which is provide along with context.

For the given company, year, and quarter, you will extract the Capex data from the provided earnings call transcript.
The data may be in tabular format within the transcript. Always give answer in millions. if u find capex as $304849 thousand then convert it to millions

### Direct Capex Inquiry:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript. Look for phrases like "capital expenditures," "CapEx," "capital spending," "investment in capital," and "capital investments."

### Contextual Capex Extraction:
Identify and extract any statements related to capital expenditures (Capex) in the earnings call transcript. This includes any mentions of planned, current, or past investments in infrastructure, technology, facilities, or equipment.

### Capex in Financial Statements:
Locate and extract the capital expenditures (Capex) figures from the financial statements section of the earnings call transcript. Look for specific numbers associated with terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."

### Capex with Time Frame:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript, including any references to the time frame (e.g., quarterly, annually) for these expenditures. Look for phrases like "capital expenditures for the quarter," "annual CapEx," and "capital spending this year."

### Capex with Specific Projects:
Identify and extract any mentions of capital expenditures (Capex) related to specific projects or initiatives in the earnings call transcript. This includes investments in new facilities, technology upgrades, or expansion projects.

### Capex in Tabular Format:
Identify and extract capital expenditures (Capex) figures presented in tabular format within the earnings call transcript. Look for tables that include terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."
for example in the below statement:
"For the fiscal year ended September 30, 2022, adjusted free cash flow of $2,977.8 million consisted of net cash
provided by operating activities of $2,703.1 million, minus capital expenditures of $496.3 million and the gains from antitrust
litigation settlements of $1.8 million, plus legal settlement payments of $772.9 million."
The capex is $496.3 million

sometime capital expenditure is gievn as in below tabular format
Capital expenditures (322,732 ) (273,407 )

If Capital Expenditures are given in the content as a range (e.g., $400M - $450M), then take the average of both. Do not give the output as a range.

### How Capex is Calculated:
Capital Expenditure (Capex) refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, buildings, or equipment. It is an important metric for understanding a company’s investment in its long-term growth.

**Formula: Capex = (Change in Net PP&E) + (Depreciation Expense)**

Where:
- Net PP&E stands for Net Property, Plant, and Equipment.
- Depreciation Expense is the allocation of the cost of tangible assets over their useful lives.

**Example Calculation:**
Let’s say a company has the following financial data for the year:
- Net PP&E at the end of the year: $500 million
- Net PP&E at the beginning of the year: $450 million
- Depreciation Expense for the year: $30 million

Using the formula:
Capex = (Net PP&E at the end of the year) - (Net PP&E at the beginning of the year) + (Depreciation Expense)
Capex = (500 - 450) + 30
Capex = 50 + 30
Capex = 80 million

So, the capital expenditure for the year is $80 million.
Do not give answers in words like (	$251101 thousand)
### Please provide the extracted information in the following format:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: [Extracted Capex Value]

If Capex data is not explicitly mentioned, indicate "NaN" for that quarter.

**Example:**
Company Name: ABC Corp
Year: 2023
Quarter: Q1
Capex: $1.2 million

Company Name: ABC Corp
Year: 2023
Quarter: Q2
Capex: NaN

Company Name: XYZ Inc
Year: 2022
Quarter: Q4
Capex: $3.5 million

If the document does not provide specific information on capital expenditure (Capex) for the company, then give the output as:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: NaN

**Context:**
{context}

**File Name:**
{file_name}

**Question:**
{question}

'''


In [9]:
import pandas as pd

def convert_to_million(value):
    if isinstance(value, str):
        value = value.replace('$', '').strip()
        if 'billion' in value:
            return float(value.replace('billion', '').strip()) * 1000
        elif 'million' in value:
            return float(value.replace('million', '').strip())
    return value

def calculate_pct_change(current, previous):
    current = convert_to_million(current)
    previous = convert_to_million(previous)

    if pd.isna(current) or pd.isna(previous):
        return 'cannot be determined'
    else:
        change = ((current - previous) / previous) * 100
        return f'increase by {change:.2f}%' if change > 0 else f'decrease by {change:.2f}%'



In [58]:
folder_path='/content/sample_data/Cencora'
#persist_directory='/content/sample_data/'
files = glob.glob(os.path.join(folder_path, '**/*.pdf'), recursive=True)
main_df=pd.DataFrame(columns=['Company Name', 'Year', 'Quarter', 'Capex'])
for fname in files:
  retriever = vectordb.as_retriever(search_kwargs={"k": 7, "filter": {"file_name": os.path.basename(fname)}})
  query = "Extract capital expendeture for given company for given quarter"
  relevant_chunks = retriever.get_relevant_documents(query)
  context = "\n".join([chunk.page_content for chunk in relevant_chunks])
  prompt = PromptTemplate(template=full_prompt, input_variables=["context",'question','file_name'])
  chain = LLMChain(llm=llm, prompt=prompt)
  result=chain.run(context=context,question=query,file_name=fname.split('/')[-1])
  lines = result.split('\n')
  my_result = {}
  for line in lines:
    try:
      key, value = line.split(': ')
    except ValueError:
      continue
    if key == "Capex":
            #value = int(value.replace('$', '').replace(',', '').split()[0])
        value = value.replace(',', '')
    elif key == "Year":
        value = int(value)
    elif key == 'Company Name':
        value = os.path.basename(fname).split('-')[0]
    my_result[key] = value

  main_df = pd.concat([main_df, pd.DataFrame(data=[my_result])], axis=0)
  main_df.replace('NaN', np.nan, inplace=True)
main_df=main_df.pivot(index=['Company Name', 'Year'], columns='Quarter', values='Capex').reset_index()
main_df.columns = ['Company Name', 'Year', 'Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']
calc_df = main_df.copy()
for quarter in ['Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']:
  #calc_df[quarter] = calc_df[quarter].replace('[\$, million]', '', regex=True)
  main_df['Q2_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q2_capex'], row['Q1_capex']), axis=1)
  main_df['Q3_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q3_capex'], row['Q2_capex']), axis=1)
  main_df['Q4_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q4_capex'], row['Q3_capex']), axis=1)



KeyboardInterrupt: 

In [39]:
main_df

,Company Name,Year,Q1_capex,Q2_capex,Q3_capex,Q4_capex,Q2_pct_change,Q3_pct_change,Q4_pct_change
0,CENCORA,2020,NaN,$144.4 million,$251.1 million,$369.677 million,cannot be determined,increase by 73.89%,increase by 47.22%
1,CENCORA,2021,$65.4 million,NaN,NaN,$438.217 million,cannot be determined,cannot be determined,cannot be determined
2,CENCORA,2022,$79.691 million,NaN,NaN,$496.3 million,cannot be determined,cannot be determined,cannot be determined
3,CENCORA,2023,$75.727 million,NaN,$282.9 million,$458.359 million,cannot be determined,cannot be determined,increase by 62.02%
4,CENCORA,2024,$74.2 million,$186.97 million,$304.849 million,NaN,increase by 151.98%,increase by 63.05%,cannot be determined


In [ ]:
# folder_path='/content/sample_data/Amazon'
# persist_directory='/content/sample_data/'
# files = glob.glob(os.path.join(folder_path, '**/*.pdf'), recursive=True)
# main_df=pd.DataFrame(columns=['Company Name', 'Year', 'Quarter', 'Capex'])
# for fname in files:
#   record=[]
#   retriever = vectordb.as_retriever(search_kwargs={"k": 5, "filter": {"file_name": os.path.basename(fname)}})
#   query = "Extract capex expendeture for given company"
#   relevant_chunks = retriever.get_relevant_documents(query)
#   context = "\n".join([chunk.page_content for chunk in relevant_chunks])
#   prompt = PromptTemplate(template=full_prompt, input_variables=["context",'question'])
#   chain = LLMChain(llm=llm, prompt=prompt)
#   result=chain.run(context=context,question=query)
#   result= result.replace(',', '').replace('\n','').split('Capex: ')[1]
#   #record.append(os.path.basename(fname).split('-')[0],os.path.basename(fname).split('-')[2],os.path.basename(fname).split('-')[1],result)
#   record.append(os.path.basename(fname).split('-')[0])
#   record.append(os.path.basename(fname).split('-')[2])
#   record.append(os.path.basename(fname).split('-')[1])
#   record.append(result)
#   main_df=pd.concat([main_df,pd.DataFrame(data=[record])],axis=0)
#   main_df.replace('NaN', np.nan, inplace=True)
# main_df=main_df.pivot(index=['Company Name', 'Year'], columns='Quarter', values='Capex').reset_index()
# main_df.columns = ['Company Name', 'Year', 'Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']
# calc_df = main_df.copy()
# for quarter in ['Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']:
#   calc_df[quarter] = calc_df[quarter].replace('[\$, million]', '', regex=True)
#   main_df['Q2_pct_change'] = calc_df.apply(lambda row: calculate_pct_change(row['Q2_capex'], row['Q1_capex']), axis=1)
#   main_df['Q3_pct_change'] = calc_df.apply(lambda row: calculate_pct_change(row['Q3_capex'], row['Q2_capex']), axis=1)
#   main_df['Q4_pct_change'] = calc_df.apply(lambda row: calculate_pct_change(row['Q4_capex'], row['Q3_capex']), axis=1)



In [ ]:
# full_prompt='''
# You are an AI assistant tasked with extracting capital expenditure (capex) data from earnings
# call transcripts. Capex refers to the funds used by a company to acquire, upgrade, and
# maintain physical assets such as property, industrial buildings, or equipment.
# It is often found in the financial statements or discussed during earnings calls.
# For the given company, year, and quarter, you will extract the capex data from
# the provided earnings call transcript.
#  The data may be in tabular format within the transcript.
#  Please provide the extracted information in the following  format:


# Capex: Extracted Capex Value

# If capex data is not explicitly mentioned, indicate "NaN" for that quarter.

# Example:

# Capex: $1.2 million
# Capex: $3.5 million
# if you think The document does not provide specific information on capital expenditure (capex) for Amazon then give below output only and do not try to give extra information.

# please stick to below output format
# Capex: NaN

# Context:
# Question:
# {question}
# '''


In [33]:
full_prompt='''
You are an AI assistant tasked with extracting capital expenditure (Capex) data from earnings call transcripts. CapEx refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, industrial buildings, or equipment. It is often found in the financial statements or discussed during earnings calls.

The name of the file has details like company name, year, and quarter. For example:
If the file name is AMZN-Q1-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q1.
If the file name is AMZN-Q2-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q2.
If the file name is CENCORA-Q2-2022-Earnings-Release, then the comany name is CENCORA, the year is 2022 and the quarter is Q2.


Always use the file name to extract company name, quarter, and year information.
Company name, quarter, and year information must be always taken from file name.

For the given company, year, and quarter, you will extract the Capex data from the provided earnings call transcript. The data may be in tabular format within the transcript.

### Direct Capex Inquiry:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript. Look for phrases like "capital expenditures," "CapEx," "capital spending," "investment in capital," and "capital investments."

### Contextual Capex Extraction:
Identify and extract any statements related to capital expenditures (Capex) in the earnings call transcript. This includes any mentions of planned, current, or past investments in infrastructure, technology, facilities, or equipment.

### Capex in Financial Statements:
Locate and extract the capital expenditures (Capex) figures from the financial statements section of the earnings call transcript. Look for specific numbers associated with terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."

### Capex with Time Frame:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript, including any references to the time frame (e.g., quarterly, annually) for these expenditures. Look for phrases like "capital expenditures for the quarter," "annual CapEx," and "capital spending this year."

### Capex with Specific Projects:
Identify and extract any mentions of capital expenditures (Capex) related to specific projects or initiatives in the earnings call transcript. This includes investments in new facilities, technology upgrades, or expansion projects.

### Capex in Tabular Format:
Identify and extract capital expenditures (Capex) figures presented in tabular format within the earnings call transcript. Look for tables that include terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."
for example in the below statement:
"For the fiscal year ended September 30, 2022, adjusted free cash flow of $2,977.8 million consisted of net cash
provided by operating activities of $2,703.1 million, minus capital expenditures of $496.3 million and the gains from antitrust
litigation settlements of $1.8 million, plus legal settlement payments of $772.9 million."
The capex is $496.3 million

sometime capital expenditure is gievn as in below tabular format
Capital expenditures (322,732 ) (273,407 )

If Capital Expenditures are given in the content as a range (e.g., $400M - $450M), then take the average of both. Do not give the output as a range.

### How Capex is Calculated:
Capital Expenditure (Capex) refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, buildings, or equipment. It is an important metric for understanding a company’s investment in its long-term growth.

**Formula: Capex = (Change in Net PP&E) + (Depreciation Expense)**

Where:
- Net PP&E stands for Net Property, Plant, and Equipment.
- Depreciation Expense is the allocation of the cost of tangible assets over their useful lives.

**Example Calculation:**
Let’s say a company has the following financial data for the year:
- Net PP&E at the end of the year: $500 million
- Net PP&E at the beginning of the year: $450 million
- Depreciation Expense for the year: $30 million

Using the formula:
Capex = (Net PP&E at the end of the year) - (Net PP&E at the beginning of the year) + (Depreciation Expense)
Capex = (500 - 450) + 30
Capex = 50 + 30
Capex = 80 million

So, the capital expenditure for the year is $80 million.
Do not give answers in words like (	$251101 thousand)
### Please provide the extracted information in the following format:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: [Extracted Capex Value]

If Capex data is not explicitly mentioned, indicate "NaN" for that quarter.

**Example:**
Company Name: ABC Corp
Year: 2023
Quarter: Q1
Capex: $1.2 million

Company Name: ABC Corp
Year: 2023
Quarter: Q2
Capex: NaN

Company Name: XYZ Inc
Year: 2022
Quarter: Q4
Capex: $3.5 million

If the document does not provide specific information on capital expenditure (Capex) for the company, then give the output as:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: NaN

**Context:**
{context}

**File Name:**
{file_name}

**Question:**
{question}

'''


In [40]:

refine_prompt = PromptTemplate.from_template(
    """
    The original question is as follows: {question}
    We have provided an existing answer: {existing_answer}
    We have the opportunity to refine the existing answer (only if needed) with
    some more context below.
    ------------
    {context}
    ------------
    Given the new context, refine the original answer to better answer the question with out changing the format.
    If the context isn't useful, return the original answer .
    """
)
question_prompt = PromptTemplate(template=full_prompt,input_variables=['context','file_name','question'])


In [42]:
refine_chain=load_qa_chain(
    llm,
    chain_type="refine",
    question_prompt=question_prompt,
    refine_prompt=refine_prompt,
    # these variables are the default values and can be modified/omitted
    document_variable_name="context",
    initial_response_name="existing_answer",
)

In [44]:

#retriever = vectordb.as_retriever(search_kwargs={"k": 7, "filter": {"file_name": os.path.basename(fname)}})
query = "Extract capital expendeture for given company for given quarter"
matching_docs=vectordb.similarity_search(query=query,k=5,filter={"file_name": os.path.basename(fname)})


**Refine chain**

In [ ]:
full_prompt='''
You are an AI assistant tasked with extracting capital expenditure (Capex) data from earnings call transcripts. CapEx refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, industrial buildings, or equipment. It is often found in the financial statements or discussed during earnings calls.

The name of the file has details like company name, year, and quarter. For example:
If the file name is AMZN-Q1-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q1.
If the file name is AMZN-Q2-2020-Earnings-Release.pdf, then the company name is AMZN, the year is 2020, and the quarter is Q2.
If the file name is CENCORA-Q2-2022-Earnings-Release, then the comany name is CENCORA, the year is 2022 and the quarter is Q2.


Always use the file name to extract company name, quarter, and year information.
Company name, quarter, and year information must be always be derived from file name which is provide along with context.

For the given company, year, and quarter, you will extract the Capex data from the provided earnings call transcript.
The data may be in tabular format within the transcript. Always give answer in millions. if u find capex as $304849 thousand then convert it to millions

### Direct Capex Inquiry:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript. Look for phrases like "capital expenditures," "CapEx," "capital spending," "investment in capital," and "capital investments."

### Contextual Capex Extraction:
Identify and extract any statements related to capital expenditures (Capex) in the earnings call transcript. This includes any mentions of planned, current, or past investments in infrastructure, technology, facilities, or equipment.

### Capex in Financial Statements:
Locate and extract the capital expenditures (Capex) figures from the financial statements section of the earnings call transcript. Look for specific numbers associated with terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."

### Capex with Time Frame:
Extract the capital expenditures (Capex) mentioned in the earnings call transcript, including any references to the time frame (e.g., quarterly, annually) for these expenditures. Look for phrases like "capital expenditures for the quarter," "annual CapEx," and "capital spending this year."

### Capex with Specific Projects:
Identify and extract any mentions of capital expenditures (Capex) related to specific projects or initiatives in the earnings call transcript. This includes investments in new facilities, technology upgrades, or expansion projects.

### Capex in Tabular Format:
Identify and extract capital expenditures (Capex) figures presented in tabular format within the earnings call transcript. Look for tables that include terms like "capital expenditures," "CapEx," "capital spending," and "investment in capital."
for example in the below statement:
"For the fiscal year ended September 30, 2022, adjusted free cash flow of $2,977.8 million consisted of net cash
provided by operating activities of $2,703.1 million, minus capital expenditures of $496.3 million and the gains from antitrust
litigation settlements of $1.8 million, plus legal settlement payments of $772.9 million."
The capex is $496.3 million

sometime capital expenditure is gievn as in below tabular format
Capital expenditures (322,732 ) (273,407 )

If Capital Expenditures are given in the content as a range (e.g., $400M - $450M), then take the average of both. Do not give the output as a range.

### How Capex is Calculated:
Capital Expenditure (Capex) refers to the funds used by a company to acquire, upgrade, and maintain physical assets such as property, buildings, or equipment. It is an important metric for understanding a company’s investment in its long-term growth.

**Formula: Capex = (Change in Net PP&E) + (Depreciation Expense)**

Where:
- Net PP&E stands for Net Property, Plant, and Equipment.
- Depreciation Expense is the allocation of the cost of tangible assets over their useful lives.

**Example Calculation:**
Let’s say a company has the following financial data for the year:
- Net PP&E at the end of the year: $500 million
- Net PP&E at the beginning of the year: $450 million
- Depreciation Expense for the year: $30 million

Using the formula:
Capex = (Net PP&E at the end of the year) - (Net PP&E at the beginning of the year) + (Depreciation Expense)
Capex = (500 - 450) + 30
Capex = 50 + 30
Capex = 80 million

So, the capital expenditure for the year is $80 million.
Do not give answers in words like (	$251101 thousand)
### Please provide the extracted information in the following format:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: [Extracted Capex Value]

If Capex data is not explicitly mentioned, indicate "NaN" for that quarter.

**Example:**
Company Name: ABC Corp
Year: 2023
Quarter: Q1
Capex: $1.2 million

Company Name: ABC Corp
Year: 2023
Quarter: Q2
Capex: NaN

Company Name: XYZ Inc
Year: 2022
Quarter: Q4
Capex: $3.5 million

If the document does not provide specific information on capital expenditure (Capex) for the company, then give the output as:
Company Name: [Company Name]
Year: [Year]
Quarter: [Q1/Q2/Q3/Q4]
Capex: NaN


**File Name:**
{file_name}

**Question:**
{question}

'''


In [19]:
folder_path='/content/sample_data/Cencora'
#persist_directory='/content/sample_data/'
files = glob.glob(os.path.join(folder_path, '**/*.pdf'), recursive=True)
main_df=pd.DataFrame(columns=['Company Name', 'Year', 'Quarter', 'Capex'])
for fname in files:
  retriever = vectordb.as_retriever(search_kwargs={"k": 7, "filter": {"file_name": os.path.basename(fname)}})
  query = "Extract capital expendeture for given company for given quarter"
  relevant_chunks = retriever.get_relevant_documents(query)
  context = "\n".join([chunk.page_content for chunk in relevant_chunks])
  file_name = os.path.basename(fname)
  question = "Extract the Capex data from the earnings call transcript."
  prompt = PromptTemplate(template=full_prompt, input_variables=['question','file_name'])
  #chain = LLMChain(llm=llm, prompt=prompt)
  chain = load_qa_chain(llm, chain_type="refine",refine_prompt=prompt)
  result = chain.invoke({
        "input_documents": relevant_chunks,
        "file_name": file_name,
        "question": question
    })
  lines = result['output_text'].split('\n')
  my_result = {}
  for line in lines:
    try:
      key, value = line.split(': ')
    except ValueError:
      continue
    if key == "Capex":
            #value = int(value.replace('$', '').replace(',', '').split()[0])
        value = value.replace(',', '')
    elif key == "Year":
        value = int(value)
    my_result[key] = value

  main_df = pd.concat([main_df, pd.DataFrame(data=[my_result])], axis=0)
  main_df.replace('NaN', np.nan, inplace=True)
main_df=main_df.pivot(index=['Company Name', 'Year'], columns='Quarter', values='Capex').reset_index()
main_df.columns = ['Company Name', 'Year', 'Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']
calc_df = main_df.copy()
for quarter in ['Q1_capex', 'Q2_capex', 'Q3_capex', 'Q4_capex']:
  #calc_df[quarter] = calc_df[quarter].replace('[\$, million]', '', regex=True)
  main_df['Q2_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q2_capex'], row['Q1_capex']), axis=1)
  main_df['Q3_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q3_capex'], row['Q2_capex']), axis=1)
  main_df['Q4_pct_change'] = main_df.apply(lambda row: calculate_pct_change(row['Q4_capex'], row['Q3_capex']), axis=1)



In [20]:
main_df

,Company Name,Year,Q1_capex,Q2_capex,Q3_capex,Q4_capex,Q2_pct_change,Q3_pct_change,Q4_pct_change
0,CENCORA,2020,NaN,NaN,NaN,NaN,cannot be determined,cannot be determined,cannot be determined
1,CENCORA,2021,NaN,NaN,NaN,NaN,cannot be determined,cannot be determined,cannot be determined
2,CENCORA,2022,NaN,NaN,NaN,NaN,cannot be determined,cannot be determined,cannot be determined
3,CENCORA,2023,NaN,NaN,NaN,NaN,cannot be determined,cannot be determined,cannot be determined
4,CENCORA,2024,NaN,NaN,NaN,NaN,cannot be determined,cannot be determined,cannot be determined


**from Azure embedding and FAISS**

In [ ]:
import os
import glob
import numpy as np
import pandas as pd
os.environ['OPENAI_API_KEY'] = '37a9009fee4f473fbc8d499ace9581fa'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-03-15-preview'
os.environ['OPENAI_API_BASE'] = 'https://new-capex.openai.azure.com/'
import json
import os



In [50]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = '37a9009fee4f473fbc8d499ace9581fa',
  api_version = "2023-03-15-preview",
  azure_endpoint ='https://new-capex.openai.azure.com/'
)

response = client.embeddings.create(
    input = "Your text string goes here",
    model= "textembedding"
)

print(response.data[0].embedding)

[-0.007578954566270113, -0.0055061643943190575, 0.011402026750147343, -0.0247525442391634, -0.024873483926057816, 0.039802949875593185, -0.010347154922783375, -0.009480412118136883, -0.013384111225605011, -0.00991042423993349, -0.011717816814780235, 0.008217253722250462, -0.014338199980556965, 0.0077603659592568874, 0.010044802911579609, -0.0049820877611637115, 0.022777177393436432, -0.0017082883277907968, 0.015319163911044598, -0.010279965586960316, 0.0049182577058672905, 0.012208298780024052, 0.004834271036088467, 0.010568879544734955, -0.006574474740773439, -0.00044596908264793456, 0.005707732401788235, -0.012853316031396389, 0.01634044200181961, 0.004454651847481728, 0.0063762660138309, -0.006856669671833515, -0.015346040017902851, -0.00615454139187932, -0.01811423897743225, 0.0037827584892511368, 0.003614785149693489, -0.01944458857178688, 0.030315818265080452, -0.006967531982809305, 0.008136626332998276, 0.009453536942601204, -0.0009347713785246015, -0.0007256445824168622, -0.007

In [82]:
import os,tiktoken
import faiss
import numpy as np
from openai import AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key='3e416ca8900e4c84bf15232375dc8c65',
    api_version="2023-03-15-preview",
    azure_endpoint='https://capexfortext.openai.azure.com/'
)

# Function to create embeddings using Azure OpenAI
def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = client.embeddings.create(
            input=text,
            model="newtextembeddings"
        )
        embeddings.append(response.data[0].embedding)
    return embeddings


def split_text(text, chunk_size=500, chunk_overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks




# /content/sample_data/Cencora/2020/CENCORA-Q1-2020-Earnings-Release.pdf
document_loader=PyPDFLoader('/content/sample_data/Cencora/2020/CENCORA-Q1-2020-Earnings-Release.pdf')
documents=document_loader.load()
text_splitter=CharacterTextSplitter(chunk_size=100,chunk_overlap=10)
#text=text_splitter.split_documents(documents)
text = ""
for doc in documents:
    text += doc.page_content

# Split text into chunks
#text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=0)
#chunks = text_splitter.split_text(text)
chunks = split_text(text)

# Create embeddings for each chunk
embeddings = create_embeddings(chunks)

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings).astype('float32')

# Initialize Faiss index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings_np)

# Save the Faiss index to disk
faiss.write_index(index, "faiss_index.bin")

print("Embeddings stored in Faiss index successfully.")

# Function to perform similarity search
def similarity_search(query, k=5):
    # Create embedding for the query
    query_embedding = create_embeddings([query])[0]
    query_embedding_np = np.array([query_embedding]).astype('float32')

    # Load the Faiss index
    index = faiss.read_index("faiss_index.bin")

    # Perform the search
    distances, indices = index.search(query_embedding_np, k)

    return distances, indices

# Example query
query = "what is the capital expenditure of a company"
distances, indices = similarity_search(query)

print("Top {} similar chunks:".format(len(indices[0])))
for i, idx in enumerate(indices[0]):
    print("Chunk {}: Distance = {}".format(idx, distances[0][i]))


Embeddings stored in Faiss index successfully.
Top 5 similar chunks:
Chunk 87: Distance = 0.427569717168808
Chunk 114: Distance = 0.4277196228504181
Chunk 100: Distance = 0.4382053017616272
Chunk 57: Distance = 0.43979543447494507
Chunk 73: Distance = 0.4454255700111389


In [83]:
chunks[87]

' )\nInventories \xa0(630,980 )\xa0(898,775 )\nAccounts payable \xa0787,037 \xa0\xa01,498,643 \xa0\nOther \xa0(233,631 )\xa0(60,668 )\nNet cash provided by operating activities \xa0142,810 \xa0\xa0478,968 \xa0\n\xa0 \xa0\xa0 \xa0\xa0\nInvesting Activities: \xa0\xa0 \xa0\xa0\nCapital expenditures \xa0(67,305 )\xa0(79,233 )\nCost of acquired companies, net of cash acquired \xa0— \xa0\xa0(52,398 )\nOther \xa04,966 \xa0\xa04,013 \xa0\nNet cash used in investing activities \xa0(62,339 )\xa0(127,618 )\n\xa0 \xa0\xa0 \xa0\xa0\nFinancing Activities: \xa0\xa0 \xa0\xa0\nNet (repayments) borrowings \xa0(13,981 )\xa08,442 \xa0\nPurchases of common sto'

In [85]:
import os
import faiss
import numpy as np
from openai import AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
import tiktoken

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key='3e416ca8900e4c84bf15232375dc8c65',
    api_version="2023-03-15-preview",
    azure_endpoint='https://capexfortext.openai.azure.com/'
)

# Function to create embeddings using Azure OpenAI
def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = client.embeddings.create(
            input=text,
            model="newtextembeddings"
        )
        embeddings.append(response.data[0].embedding)
    return embeddings

# Load and split the document
file_path = '/content/sample_data/Cencora/2020/CENCORA-Q1-2020-Earnings-Release.pdf'
document_loader = PyPDFLoader(file_path)
documents = document_loader.load()
text = ""
for doc in documents:
    text += doc.page_content

# Manually split text into chunks
def split_text(text, chunk_size=500, chunk_overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

chunks = split_text(text)

# Ensure each chunk is within the token limit


# Create embeddings for each chunk and add metadata
embeddings = create_embeddings(chunks)
metadata = [{'file_name': os.path.basename(file_path)} for _ in chunks]

# Convert embeddings to numpy array
embeddings_np = np.array(embeddings).astype('float32')

# Initialize Faiss index
dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
index.add(embeddings_np)

# Save the Faiss index and metadata to disk
faiss.write_index(index, "faiss_index.bin")
np.save("metadata.npy", metadata)

print("Embeddings and metadata stored successfully.")

# Function to perform similarity search
def similarity_search(query, file_name, k=5):
    # Create embedding for the query
    query_embedding = create_embeddings([query])[0]
    query_embedding_np = np.array([query_embedding]).astype('float32')

    # Load the Faiss index and metadata
    index = faiss.read_index("faiss_index.bin")
    metadata = np.load("metadata.npy", allow_pickle=True)

    # Perform the search
    distances, indices = index.search(query_embedding_np, k)

    # Filter results by file_name
    filtered_results = [(dist, idx) for dist, idx in zip(distances[0], indices[0]) if metadata[idx]['file_name'] == file_name]

    return filtered_results

# Example query
query = "please extract capex data"
file_name = os.path.basename(file_path)
results = similarity_search(query, file_name)

print("Top {} similar chunks:".format(len(results)))
for i, (dist, idx) in enumerate(results):
    print("Chunk {}: Distance = {}".format(idx, dist))


Embeddings and metadata stored successfully.
Top 5 similar chunks:
Chunk 100: Distance = 0.4359811544418335
Chunk 114: Distance = 0.4398251175880432
Chunk 87: Distance = 0.4420127868652344
Chunk 91: Distance = 0.4514691233634949
Chunk 106: Distance = 0.45371657609939575


In [88]:
import os
import numpy as np
from openai import AzureOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.docstore.document import Document
import tiktoken
import faiss

# Initialize Azure OpenAI client
client = AzureOpenAI(
    api_key='3e416ca8900e4c84bf15232375dc8c65',
    api_version="2023-03-15-preview",
    azure_endpoint='https://capexfortext.openai.azure.com/'
)

# Function to create embeddings using Azure OpenAI
def create_embeddings(texts):
    embeddings = []
    for text in texts:
        response = client.embeddings.create(
            input=text,
            model="newtextembeddings"
        )
        embeddings.append(response.data[0].embedding)
    return embeddings

# Load and split the document
file_path = '/content/sample_data/Cencora/2020/CENCORA-Q1-2020-Earnings-Release.pdf'
document_loader = PyPDFLoader(file_path)
documents = document_loader.load()
text = ""
for doc in documents:
    text += doc.page_content

# Manually split text into chunks
def split_text(text, chunk_size=500, chunk_overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - chunk_overlap
    return chunks

chunks = split_text(text)

# Ensure each chunk is within the token limit

# Create embeddings for each chunk and add metadata
embeddings = create_embeddings(chunks)
metadata = [{'file_name': os.path.basename(file_path)} for _ in chunks]

# Create documents with metadata
documents = [Document(page_content=chunk, metadata=meta) for chunk, meta in zip(chunks, metadata)]

# Initialize Faiss index
dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

# Initialize InMemoryDocstore
docstore = InMemoryDocstore()

# Initialize FAISS vector store
vector_store = FAISS(index=index, docstore=docstore, index_to_docstore_id={},embedding_function=create_embeddings)

# Add documents to the vector store
vector_store.add_documents(documents)

# Save the Faiss index and metadata to disk
vector_store.save_local("faiss_index")

print("Embeddings and metadata stored successfully.")

# Function to perform similarity search
def similarity_search(query, file_name, k=5):
    # Create embedding for the query
    query_embedding = create_embeddings([query])[0]

    # Load the Faiss index and metadata
    vector_store = FAISS.load_local("faiss_index", embedding_function=create_embeddings)

    # Perform the search
    results = vector_store.similarity_search_with_score(query, k=k)

    # Filter results by file_name
    filtered_results = [(score, doc) for score, doc in results if doc.metadata['file_name'] == file_name]

    return filtered_results

# Example query
query = "Your query text goes here"
file_name = os.path.basename(file_path)
results = similarity_search(query, file_name)

print("Top {} similar chunks:".format(len(results)))
for i, (score, doc) in enumerate(results):
    print("Chunk {}: Score = {}".format(i, score))


KeyboardInterrupt: 